In [36]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 37, Finished, Available, Finished)

In [37]:
df_sales = pd.read_excel("abfss://Ecommerce@onelake.dfs.fabric.microsoft.com/Ecommere_Lh.Lakehouse/Files/Sales_01012023.xlsx", sheet_name="Sales")
display(df_sales)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 38, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0c0a0f02-4d6e-4c9e-85db-84170210b027)

In [38]:
df1 = spark.createDataFrame(df_sales)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 39, Finished, Available, Finished)

In [39]:
display(df1)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 40, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, be152405-de45-4e11-97ea-1e9fe99291e8)

In [40]:
df_returns = pd.read_excel("abfss://Ecommerce@onelake.dfs.fabric.microsoft.com/Ecommere_Lh.Lakehouse/Files/Sales_01012023.xlsx", sheet_name="Returns")
display(df_returns)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 41, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 797fdd78-8a52-4f73-a994-b728e08941bb)

In [41]:
df2 = spark.createDataFrame(df_returns)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 42, Finished, Available, Finished)

In [42]:
display(df2)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 55963a3e-ca6e-47c3-a26b-0739615b88cc)

In [43]:
display(df2.head(10))

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 44, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7bf0dcf2-c55a-4e53-a971-be63623cffb9)

In [50]:
df_final = df1.join(df2, df1.Order_ID == df2.Order_ID, how="left").drop(df2.Order_ID, df2.Customer_Name, df2.Sales_Amount)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 51, Finished, Available, Finished)

In [54]:
df_mod = df_final.withColumns({"Order_Year": year("Order_Date"), \
                    "Order_Month": month("Order_Date"), \
                    "Created_TS": current_timestamp(), \
                    "Modified_TS": current_timestamp(), \
                    })

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 55, Finished, Available, Finished)

In [55]:
df_mod.createOrReplaceTempView("ViewSales")

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 56, Finished, Available, Finished)

In [60]:
%%sql
create table if not EXISTS Ecommere_Lh.Bronze_sales
(
    Order_ID string,
    Order_Date date,
    Shipping_Date date,
    Aging int,
    Ship_Mode string,
    Product_category string,
    Product string,
    Sales float,
    Quantity float,
    Discount float,
    Profit float,
    Shipping_cost float,
    Order_priority string,
    Cusotmer_ID string,
    Customer_Name string,
    Segment string,
    City string,
    State string,
    Country string,
    Region string,
    Return string,
    Order_year int,
    Order_month int,
    Created_TS TIMESTAMP,
    Modified_TS timestamp
)
using DELTA
PARTITIONED by(Order_year, Order_Month)

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 61, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [62]:
%%sql
Merge into Ecommere_Lh.Bronze_sales as BS
using ViewSales as VS 
On BS.Order_year = VS.Order_Year and BS.Order_month = VS.Order_Month and BS.Order_ID=VS.Order_ID
when matched then
update set
    BS.Order_Date = VS.Order_Date,
    BS.Shipping_Date = VS.Shipping_Date,
    BS.Aging  = VS.Aging,
    BS.Ship_Mode  = VS.Ship_Mode,
    BS.Product_category  = VS.Product_Category,
    BS.Product string  = VS.Product,
    BS.Sales  = VS.Sales,
    BS.Quantity  = VS.Quantity,
    BS.Discount  = VS.Discount,
    BS.Profit  = VS.Profit,
    BS.Shipping_cost  = VS.Shipping_Cost,
    BS.Order_priority  = VS.Order_Priority,
    BS.Cusotmer_ID  = VS.Customer_ID,
    BS.Customer_Name  = VS.Customer_Name,
    BS.Segment  = VS.Segment,
    BS.City  = VS.City,
    BS.State  = VS.State,
    BS.Country  = VS.Country,
    BS.Region  = VS.Region,
    BS.Return  = VS.Return,
    BS.Order_year  = VS.Order_Year,
    BS.Order_month  = VS.Order_Month,
    Created_TS = VS.Created_TS,
    Modified_TS = VS.Modified_TS
    )

StatementMeta(, 928eb7a9-9efa-4c6b-9f49-42f3503d1811, 63, Finished, Available, Finished)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'string': extra input 'string'.(line 11, pos 15)

== SQL ==
Merge into Ecommere_Lh.Bronze_sales as BS
using ViewSales as VS 
On BS.Order_year = VS.Order_Year and BS.Order_month = VS.Order_Month and BS.Order_ID=VS.Order_ID
when matched then
update set
    BS.Order_Date = VS.Order_Date,
    BS.Shipping_Date = VS.Shipping_Date,
    BS.Aging  = VS.Aging,
    BS.Ship_Mode  = VS.Ship_Mode,
    BS.Product_category  = VS.Product_Category,
    BS.Product string  = VS.Product,
---------------^^^
    BS.Sales  = VS.Sales,
    BS.Quantity  = VS.Quantity,
    BS.Discount  = VS.Discount,
    BS.Profit  = VS.Profit,
    BS.Shipping_cost  = VS.Shipping_Cost,
    BS.Order_priority  = VS.Order_Priority,
    BS.Cusotmer_ID  = VS.Customer_ID,
    BS.Customer_Name  = VS.Customer_Name,
    BS.Segment  = VS.Segment,
    BS.City  = VS.City,
    BS.State  = VS.State,
    BS.Country  = VS.Country,
    BS.Region  = VS.Region,
    BS.Return  = VS.Return,
    BS